In [1]:
import random
from random import choice
import numpy as np
import pandas as pd
from transformers import BertTokenizer,  BertConfig, TFBertModel
import tensorflow as tf
from tensorflow.keras import backend as K
import os

# 数据集整理
## 读取数据

In [20]:
def json_to_df(path, nrows=False):
    if nrows:
        df = pd.read_json(path, nrows=nrows, lines=True)
    else:
        df = pd.read_json(path, lines=True)
    df = df[['text', 'spo_list']]
    return df

def merge_df(dir_path):
    total_df = pd.DataFrame()
    for fn in os.listdir(dir_path):
        df = json_to_df(os.path.join(dir_path, fn))
        df_fn = fn[:fn.rfind('.')]
        df.insert(0, 'fn', df_fn)
        total_df =  total_df.append(df)
    total_df.reset_index(drop=True, inplace=True)
    print(f'data size: {total_df.shape}') #
    print(f'data sample: {df.sample(5)}')
    return total_df   

def read_schemads(path_or_df):
    if not isinstance(path_or_df, pd.DataFrame):
        print(1)
        schemads_path = path_or_df
        predicate_data = pd.read_json(schemads_path, lines=True)
        id2p = predicate_data['predicate'].drop_duplicates().reset_index(drop=True).to_dict()
    else:
        df = path_or_df
        id2p = df['spo_list'].apply(lambda spo_list: [spo['predicate'] for spo in spo_list])
        id2p = id2p.explode().drop_duplicates().reset_index(drop=True).to_dict()
    p2id = dict(zip(id2p.values(), id2p.keys()))
    print(f'length of p2id :{len(p2id)}')#
    print(f'random p2id sample:{random.sample(p2id.items(), 5)}')#
    return id2p, p2id

In [21]:
# %%time
# # 百度三元组关系数据集
# train_path ='../data/百度关系抽取数据集/train_data.json'
# train_data = json_to_df(train_path, nrows=10000)
# print(f'Train data size: {train_data.shape}') #

# dev_path = '../data/百度关系抽取数据集/dev_data.json'
# dev_data = json_to_df(dev_path, nrows=5000)
# print(f'Validation data size: {dev_data.shape}') 

# schemads_path = '../data/百度关系抽取数据集/all_50_schemas'
# id2p, p2id = read_schemads(schemads_path)#

In [22]:
## 招股说明书三元组数据集
dir_path = '../data/招股说明书三元组数据集'
train_size=0.9
df = merge_df(dir_path)

train_data = df.sample(frac=train_size,random_state=200)
dev_data = df.drop(train_data.index).reset_index(drop=True)
# schemads_path = '../data/招股说明书三元组数据集/all_50_schemas'
id2p, p2id = read_schemads(df)

data size: (6093, 3)
data sample:                   fn                                               text  \
3   阿科力首次公开发行股票招股说明书  2013年6月3日，阿科力有限股东会做出决议，同意阿科力有限整体变更为股份有限公司，公司整体...   
46  阿科力首次公开发行股票招股说明书  2001年7月11日，经公司股东会决议批准，将原注册资本50万元增资至200万元，各股东资金...   
86  阿科力首次公开发行股票招股说明书  2017年1-6月经营活动现金流量净额为1,990.65万元，2016年1-6月经营活动现金...   
4   阿科力首次公开发行股票招股说明书  2015年5月26日，无锡诚鼎创业投资中心（有限合伙）向上海中汇金玖三期创业投资基金合伙企业...   
84  阿科力首次公开发行股票招股说明书  报告期内，公司管理费用主要由职工薪酬、研发费用、环境保护费、中介服务费等项目构成。2015年...   

                                             spo_list  
3   [{'predicate': '变更日期', 'object_type': '日期', 's...  
46  [{'predicate': '注册资本', 'object_type': '金额', 's...  
86  [{'predicate': '公司经营活动产生的现金流量净额', 'object_type...  
4   [{'predicate': '转让日期', 'object_type': '日期', 's...  
84  [{'predicate': '管理费用', 'object_type': '金额', 's...  
length of p2id :114
random p2id sample:[('其他流动资产', 65), ('速动比率', 18), ('非流动负债', 107), ('每股净资产', 50), ('应收账款周转率', 73)]


## 清洗数据

In [23]:
def clean_spo(spo_list):
    for spo in spo_list:
        spo['predicate'] = spo['predicate'].lower()
        spo['subject'] = spo['subject'].lower()
        spo['object'] = spo['object'].lower()
    return spo_list

def data_clean(df):
    df['text'] = df['text'].str.lower()
    df['spo_list'] = df['spo_list'].apply(clean_spo)
    return df

In [24]:
train_data = data_clean(train_data)
dev_data = data_clean(dev_data)

train_text = train_data['text'].to_list()
train_spo = train_data['spo_list'].to_list()

spo_single_count = train_data['spo_list'].apply(lambda x: len(x))
spo_count = spo_single_count.sum()
print('spo_count', spo_count)
                                                
dev_text = dev_data['text'].to_list()
dev_spo = dev_data['spo_list'].to_list()

spo_count 15412


## 数据集处理

In [25]:
def find_head_idx(pattern, sequence):
    """从sequence中寻找子串pattern
    如果找到，返回第一个下标；否则返回-1。
    """
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1

In [26]:
def data_process(df, p2id, tokenizer, max_length):
    long_text_count = 0
    total_token_ids, total_token_type_ids, total_attention_mask = [], [], []
    total_subject_labels, total_subject_ids, total_object_labels = [], [], []
    for idx in df.index:
        row = df.loc[idx]
        inputs = tokenizer(row['text'], max_length=max_length, padding='max_length', truncation=True)
        token_ids, token_type_ids, attention_mask = inputs['input_ids'], inputs['token_type_ids'], inputs['attention_mask']
        # 实体关系token id 位置字典
        s2op_map = {}
        for sop_n, spo in enumerate(row['spo_list']):
            sub_ids = tokenizer.encode(spo['subject'])[1:-1]
            p_id = p2id[spo['predicate']]
            obj_ids = tokenizer.encode(spo['object'])[1:-1]
            # 查找subject 和 object对应的token id 起始索引
            sub_head_idx = find_head_idx(sub_ids, inputs['input_ids'])
            obj_head_idx = find_head_idx(obj_ids, inputs['input_ids'])
            if sub_head_idx != -1 and obj_head_idx != -1:
                # 获取subject 起始位置的索引和结束位置索引的元组
                sub = (sub_head_idx, sub_head_idx + len(sub_ids) - 1)
                # 获取object 起始位置的索引和结束位置索引元组以及与关系标签id的元组对
                obj = (obj_head_idx, obj_head_idx + len(obj_ids) - 1, p_id)
                # print('---- sub -----', sub)#    
                if sub not in s2op_map:
                    s2op_map[sub] = []
                s2op_map[sub].append(obj)
                # print('-----------', s2op_map)#                
                {(22,23):[(28, 29, 7), (25, 26, 8)]}  
            else:
                long_text_count +=1
                # print('--idx--', idx, '--text--', row['text'])
                # print('--sop_n--', sop_n, '--spo--', spo, '\n')


        if s2op_map:
        # subject标签
            subject_labels = np.zeros((max_length, 2))
            for s in s2op_map:
                #sub_head
                subject_labels[s[0], 0] = 1
                #sub_tail
                subject_labels[s[1], 1] = 1
            # 随机选一个subject
            sub_head, sub_tail = choice(list(s2op_map.keys()))
            subject_ids = (sub_head, sub_tail)
            # sub_head, sub_tail = np.array(list(s2op_map.keys())).T
            # sub_head = np.random.choice(sub_head)
            # sub_tail = np.random.choice(sub_tail[sub_tail >= sub_head])
            # 对应的object标签
            object_labels = np.zeros((len(token_ids), len(p2id), 2))
            for op in s2op_map.get((sub_head, sub_tail), []):
                # print(op)
                # obj_head
                object_labels[op[0], op[2], 0] = 1
                # obj_tail
                object_labels[op[1], op[2], 1] = 1

            # 所有数据汇总
            total_token_ids.append(token_ids)
            total_token_type_ids.append(token_type_ids)
            total_attention_mask.append(attention_mask)
            total_subject_labels.append(subject_labels)
            total_subject_ids.append(subject_ids)
            total_object_labels.append(object_labels)  
    print('long_text_count', long_text_count)                      
    return total_token_ids, total_token_type_ids, total_attention_mask, \
           total_subject_labels, total_subject_ids, total_object_labels

In [27]:
# input_demo = data_process(train_data[3:4], p2id, tokenizer, max_length)
# print(np.array(input_demo[5]).shape)
# print(train_spo[3])
# print(np.array(input_demo[5])[0, 25, 14, 0], np.array(input_demo[5])[0, 27, 14, 1])
# print(np.array(input_demo[5])[0, 14, 17, 0], np.array(input_demo[5])[0, 14, 17, 1])
# # tokenizer.decode(train_input_ids[3][op_s:op_e+1])
# print(train_spo[3])

In [28]:
%%time
model_path = '../model_dirs/bert-base-chinese'  
tokenizer = BertTokenizer.from_pretrained(model_path)
max_length = 512
train_input_ids, train_token_type_ids, train_attention_mask, train_subject_labels, train_subject_ids, train_object_labels  = data_process(train_data, p2id, tokenizer, max_length)
print(train_subject_ids[:2])
print(np.array(train_subject_labels).shape)

long_text_count 126
[(3, 4), (1, 6)]
(5275, 512, 2)
Wall time: 20.2 s


In [29]:
%%time
val_inputs = tokenizer(dev_text, max_length=max_length, padding='max_length', truncation=True, return_tensors='tf') 
val_input_ids, val_attention_mask = val_inputs['input_ids'], val_inputs['attention_mask']

Wall time: 621 ms


In [30]:
ix = 1178
text = train_data.loc[ix,'text']
print(len(text), text)
sub = train_data.loc[ix,'spo_list'][0]['subject']
obj = train_data.loc[ix,'spo_list'][0]['object']
print(sub, obj)
max_length = 256
s_ids = tokenizer.encode(sub, max_length=max_length, truncation=True)[1:-1]
o_ids = tokenizer.encode(obj, max_length=max_length, truncation=True)[1:-1]
text_ids = tokenizer.encode(text, max_length=max_length, padding='max_length', truncation=True)
print(text_ids)
print(s_ids)
print(o_ids)
find_head_idx(s_ids, text_ids), find_head_idx(o_ids, text_ids)

87 福州三期epc项目、洛碛epc项目和汕尾二期epc项目受土建工程及设备到货安装进度影响，2018年第四季度合计确认收入14,603.42万元，高于本年度其他季度收入确认金额；
2018年第四季度 14,603.42万元
[101, 4886, 2336, 676, 3309, 10676, 8177, 7555, 4680, 510, 3821, 4816, 10676, 8177, 7555, 4680, 1469, 3730, 2227, 753, 3309, 10676, 8177, 7555, 4680, 1358, 1759, 2456, 2339, 4923, 1350, 6392, 1906, 1168, 6573, 2128, 6163, 6822, 2428, 2512, 1510, 8024, 8271, 2399, 5018, 1724, 2108, 2428, 1394, 6369, 4802, 6371, 3119, 1057, 8122, 117, 13277, 119, 8239, 674, 1039, 8024, 7770, 754, 3315, 2399, 2428, 1071, 800, 2108, 2428, 3119, 1057, 4802, 6371, 7032, 7583, 8039, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

(42, 54)

In [31]:
## debug
model_path = '../model_dirs/bert-base-chinese'  
tokenizer = BertTokenizer.from_pretrained(model_path)
max_length = 128
print(f"text ---\n {train_data.loc[47:100,'text']}")
inputs = tokenizer(train_data.loc[47:100,'text'].to_list(), max_length=max_length, padding='max_length', return_tensors='tf', truncation=True)
print('inputs keys --\n', inputs.keys())
print(f"input_ids --\n {inputs['input_ids']}")
tokens = tokenizer.decode(inputs['input_ids'][0])
print('tokens --\n', tokens)
print(f"spo_list ------\n {train_data.loc[47,'spo_list']}")
text_len = len(tokens)
text_len
bert_model = TFBertModel.from_pretrained(model_path, output_hidden_states=True)
outputs = bert_model(inputs)
last_hidden_state, pooler_output, hidden_states = outputs[:3]

text ---
 47      当前，信息技术和信息化的不断发展和普及促使全球互联网渗透率不断提升，由此扩大了网络用户的数量...
2466    2017年11月3日，上海市徐汇区商务委员会出具《外商投资企业变更备案回执》（沪徐外资备20...
5399    2002年8月，由本公司与控股子公司上海博佳投资管理有限公司共同投资组建的台州市路桥至泽国至...
6022    所以，公司根据未来集中精力发展壮大聚醚胺和光学材料的产品发展的战略规划，经过审慎决策和过渡期...
1787    2019年末，公司库存商品账面价值为8,654.92万元，较2018年末有所增长，主要系20...
                              ...                        
4299    2017年9月，成燃有限整体变更为股份公司，成都燃气选举产生了股份公司的新一届管理层。201...
4116    公司以1999年9月30日资产评估基准日的资产状况，于1999年12月正式移交建账，根据江西...
3309    截至本招股说明书出具日，新疆生产建设兵团国有资产经营公司持有新疆生产建设兵团供销合作总公司1...
1071    2010年6月30日，重钢集团与重钢环保签订股权转让协议，重钢集团将其持有的三峰卡万塔60%...
100     2019年度，公司信用减值损失回转204.09万元，主要由于公司根据财政部\n的规定执行《企...
Name: text, Length: 1779, dtype: object
inputs keys --
 dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
input_ids --
 [[ 101 2496 1184 ... 5635 8109  102]
 [ 101 8109 2399 ...    0    0    0]
 [ 101 8301 2399 ...  119 8360  102]
 ...
 [ 101 2779 5635 ...    0    0    0]
 [ 101 8166 2399 ...    0    0    0]
 [ 101 9160 2399 ...    0    0 

Some layers from the model checkpoint at ../model_dirs/bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ../model_dirs/bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
hidden_states[-1].shape

TensorShape([54, 128, 768])

In [ ]:
def extract_subject(output, subject_ids):
    start = tf.gather(output, subject_ids[:,:1], batch_dims=1)
    end = tf.gather(output, subject_ids[:,1:], batch_dims=1)
    subject = tf.keras.layers.Concatenate(axis=2)([start, end])
    return subject[:, 0]

extract_subject(hidden_states[-1], tf.constant(train_subject_ids[:hidden_states[-1].shape[0]]))

<tf.Tensor: shape=(54, 1536), dtype=float32, numpy=
array([[ 0.25873297,  0.49238703,  0.63826686, ...,  0.11304718,
        -0.9661733 , -0.4333823 ],
       [ 0.42852157, -0.17608842,  0.8069094 , ...,  1.4964044 ,
        -0.49943933, -0.20207542],
       [ 0.5664912 , -0.12230435,  0.19252312, ...,  0.5424045 ,
         0.27006415,  0.08944741],
       ...,
       [ 0.46302688, -0.4424666 ,  0.5141795 , ...,  0.4153874 ,
         0.5806328 ,  0.04473295],
       [-0.11299253, -0.3138919 , -1.4666147 , ...,  0.36222684,
         0.43323463,  0.29208207],
       [ 0.6611088 ,  0.7782924 , -0.7245413 , ...,  0.38568816,
         0.23528749,  0.6906782 ]], dtype=float32)>

In [ ]:
class LayerNormalization(tf.keras.layers.Layer):
    """(Conditional) Layer Normalization
    hidden_*系列参数仅为有条件输入时(conditional=True)使用
    """
    def __init__(
        self,
        center=True,
        scale=True,
        epsilon=None,
        conditional=False,
        hidden_units=None,
        hidden_activation='linear',
        hidden_initializer='glorot_uniform',
        **kwargs):
        super(LayerNormalization, self).__init__(**kwargs)
        self.center = center
        self.scale = scale
        self.conditional = conditional
        self.hidden_units = hidden_units
        self.hidden_activation = tf.keras.activations.get(hidden_activation)
        self.hidden_initializer = tf.keras.initializers.get(hidden_initializer)
        self.epsilon = epsilon or 1e-12
        
    def compute_mask(self, inputs, mask=None):
        if self.conditional:
            masks = mask if mask is not None else []
            masks = [m[None] for m in masks if m is not None]
            if len(masks) == 0:
                return None
            else:
                return K.all(K.concatenate(masks, axis=0), axis=0)
        else:
            return mask
        
    def build(self, input_shape):
        super(LayerNormalization, self).build(input_shape)
        if self.conditional:
            shape = (input_shape[0][-1],)
        else:
            shape = (input_shape[-1],)
        if self.center:
            self.beta = self.add_weight(
                shape=shape, initializer='zeros', name='beta')
        if self.scale:
            self.gamma = self.add_weight(
                shape=shape, initializer='ones', name='gamma')
        if self.conditional:
            if self.hidden_units is not None:
                self.hidden_dense = tf.keras.layers.Dense(
                    units=self.hidden_units,
                    activation=self.hidden_activation,
                    use_bias=False,
                    kernel_initializer=self.hidden_initializer)
            if self.center:
                self.beta_dense = tf.keras.layers.Dense(
                    units=shape[0], use_bias=False, kernel_initializer='zeros')
            if self.scale:
                self.gamma_dense = tf.keras.layers.Dense(
                    units=shape[0], use_bias=False, kernel_initializer='zeros')

    def call(self, inputs):
        """如果是条件Layer Norm，则默认以list为输入，第二个是condition
        """
        if self.conditional:
            inputs, cond = inputs
            if self.hidden_units is not None:
                cond = self.hidden_dense(cond)
            for _ in range(K.ndim(inputs) - K.ndim(cond)):
                cond = K.expand_dims(cond, 1)
            if self.center:
                beta = self.beta_dense(cond) + self.beta
            if self.scale:
                gamma = self.gamma_dense(cond) + self.gamma
        else:
            if self.center:
                beta = self.beta
            if self.scale:
                gamma = self.gamma
        outputs = inputs
        if self.center:
            mean = K.mean(outputs, axis=-1, keepdims=True)
            outputs = outputs - mean
        if self.scale:
            variance = K.mean(K.square(outputs), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            outputs = outputs / std
            outputs = outputs * gamma
        if self.center:
            outputs = outputs + beta
        return outputs        

In [ ]:

def E2EModel(pretrained_path, max_length, p2id):
    input_ids = tf.keras.layers.Input((max_length,), dtype=tf.int32, name='input_ids')
    token_type_ids = tf.keras.layers.Input((max_length,), dtype=tf.int32, name='total_segment_ids')
    attention_mask = tf.keras.layers.Input((max_length,), dtype=tf.int32, name='attention_mask')
    subject_ids = tf.keras.layers.Input((2,), dtype=tf.int32, name='subject_ids')

    bert_model = TFBertModel.from_pretrained(pretrained_path, output_hidden_states=True)
    outputs = bert_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
    last_hidden_state, pooler_output, hidden_states = outputs[:3]
    layer_1 = hidden_states[-1]
    
    subject_preds = tf.keras.layers.Dense(units=2, activation='sigmoid',)(layer_1)
    subject_preds = tf.keras.layers.Lambda(lambda x: x**2)(subject_preds)
    subject_model = tf.keras.models.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=subject_preds)

    subject = extract_subject([layer_1, subject_ids])
    Normalization_1 = LayerNormalization(conditional=True)([layer_1, subject])
    output = tf.keras.layers.Dense(units=len(p2d) * 2, activation='sigmoid' )(output)
    output = tf.keras.layers.Lambda(lambda x: x**4)(output)
    object_preds = tf.reshape((-1, len(p2id), 2))(output)
    object_model = tf.keras.models.Model(input=[input_ids, token_type_ids, attention_mask, subject_ids], outputs=object_preds)

    train_model = tf.keras.models.Model(input=[input_ids, token_type_ids, attention_mask, subject_ids], outputs= [subject_preds, object_preds])


In [1]:
import os
pid = os.getpid()
!kill -9 $pid
!nvidia-smi

'kill' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


Tue Jun 22 21:06:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 466.27       Driver Version: 466.27       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   43C    P8    N/A /  N/A |    173MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------